In [ ]:
#!pip install geopy

In [ ]:
#!pip install geocoder

In [ ]:
#!pip install uszipcode

In [ ]:
#!pip install arcgis

In [ ]:
#!pip install --force-reinstall numpy==1.23.3

In [ ]:
#import numpy
#numpy.version.version

In [3]:
!pip install numba

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit, KFold, cross_val_score, GridSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.cluster import KMeans
#import geopy
#from geopy.geocoders import Nominatim
#from arcgis.geocoding import reverse_geocode


UsageError: Line magic function `%%timeit` not found.


# Importing & Exploring Data

In [ ]:
df_crashes = pd.read_csv('data/Traffic_Crashes_-_Crashes.csv')
df_crashes.head()

In [ ]:
df_crashes.info()

In [ ]:
df_crashes.TRAFFIC_CONTROL_DEVICE.value_counts()

In [ ]:
df_crashes.isna().sum()

In [ ]:
# Keeping relevant Features
df_crashes_drop = df_crashes [[
    'CRASH_RECORD_ID',
#    'RD_NO',
    'CRASH_DATE',
    'POSTED_SPEED_LIMIT',
    'WEATHER_CONDITION',
#    'LIGHTING_CONDITION',
#    'FIRST_CRASH_TYPE',
    'ROADWAY_SURFACE_COND',
    'ROAD_DEFECT',
#    'CRASH_TYPE',
#    'DAMAGE',
#    'PRIM_CONTRIBUTORY_CAUSE',
#    'STREET_NAME',
#    'NUM_UNITS',
    'INJURIES_TOTAL',
    'INJURIES_FATAL',
    'CRASH_HOUR',
    'CRASH_DAY_OF_WEEK',
    'CRASH_MONTH',
    'LATITUDE',
    'LONGITUDE',
#    'LOCATION',
]]
print(df_crashes_drop.shape)

In [ ]:
df_crashes_drop.describe()

In [ ]:
# Dropping Rows with Latitude & Longitude = 0
df_crashes_drop = df_crashes_drop[df_crashes_drop['LATITUDE'] != 0]

In [ ]:
df_crashes_drop['has_injuries'] = df_crashes_drop.INJURIES_TOTAL.apply(lambda x: int(x>0))
df_crashes_drop['has_fatality'] = df_crashes_drop.INJURIES_FATAL.apply(lambda x: int(x>0))

In [ ]:
# Visualizing the distribution of accidents by having injuries ( Alternatively we can get the same for fatality)
crash_df_ = df_crashes_drop.groupby(by=['LONGITUDE','LATITUDE']).agg(crashes=('has_injuries', 'count'), has_injuries=('has_injuries', 'max')).reset_index()
crash_df_

In [ ]:
plt.subplots(figsize=(10,10))
fg = sns.scatterplot(data=crash_df_, x='LONGITUDE',y='LATITUDE',
                hue='has_injuries', sizes={1:1,25:12.5,50:25,100:50,200:100}, size_norm=(1,100), size='crashes', alpha=.4)
fg.set_axis_off()
fg.set_title('Crashes in Chicago',fontweight='bold');

In [ ]:
df_crashes_drop.replace({'UNKNOWN':np.nan}, inplace=True)

In [ ]:
df_crashes_ = df_crashes_drop.dropna(subset = 'LATITUDE', axis = 0);

In [ ]:
df_vehicles = pd.read_csv('data/Traffic_Crashes_-_Vehicles.csv')
df_vehicles.head()

In [ ]:
df_vehicles.info()

In [ ]:
# Keeping Relevant Features
df_vehicles_drop = df_vehicles [[
    'CRASH_RECORD_ID',
#    'RD_NO',
#    'CRASH_DATE',
    'VEHICLE_ID',
#    'MAKE',
#    'MODEL',
#    'LIC_PLATE_STATE',
    'VEHICLE_YEAR',
#    'VEHICLE_DEFECT',
#    'VEHICLE_TYPE',
#    'VEHICLE_USE',
#    'TRAVEL_DIRECTION',
    'OCCUPANT_CNT',
#    'VEHICLE_CONFIG',
]]
print(df_vehicles_drop.shape)

In [ ]:
df_vehicles.VEHICLE_CONFIG.value_counts()

In [ ]:
df_vehicles_drop.describe()

In [ ]:
# Dropping rows with vehicle year bigger than 2022 and below 1970
df_vehicles_drop1 = df_vehicles_drop[(df_vehicles_drop['VEHICLE_YEAR'] >= 1970) & (df_vehicles_drop['VEHICLE_YEAR'] <= 2022) | (df_vehicles_drop['VEHICLE_YEAR'].isna())]

In [ ]:
pd.set_option('display.max_rows', 100)
df_vehicles_drop1.isna().sum()

In [ ]:
pd.set_option('display.max_rows', 100)
df_vehicles_drop1.head()

In [ ]:
# Dropping Vehicles with 0 Occupant (Parked Cars)
df_vehicles_w_occ = df_vehicles_drop1[df_vehicles_drop.OCCUPANT_CNT != 0.0]
print(df_vehicles_w_occ.shape)

In [ ]:
df_vehicles_w_occ.isna().sum()

In [ ]:
# Replacing all UNKNOWN values to missing values
df_vehicles_w_occ.replace({'UNKNOWN':np.nan}, inplace=True)

In [ ]:
df_people = pd.read_csv('data/Traffic_Crashes_-_PEOPLE.csv')
df_people.head()


In [ ]:
pd.set_option('display.max_rows', 100)
df_people.isna().sum()

In [ ]:
# Keeping Relevant Features
df_people_drop = df_people [[
    'CRASH_RECORD_ID',
#    'RD_NO',
#    'PERSON_ID',
    'PERSON_TYPE',
    'VEHICLE_ID',
#    'CITY',
#    'STATE',
#    'ZIPCODE',
    'SEX',
    'AGE',
#    'DRIVERS_LICENSE_STATE',
#    'DRIVERS_LICENSE_CLASS',
    'SAFETY_EQUIPMENT',
    'AIRBAG_DEPLOYED',
#    'EJECTION',
#    'INJURY_CLASSIFICATION',
#    'DRIVER_VISION',
#    'DRIVER_ACTION',
#    'PHYSICAL_CONDITION',
#    'PEDPEDAL_ACTION',
#    'PEDPEDAL_VISIBILITY',
#    'PEDPEDAL_LOCATION',
#    'BAC_RESULT',
#    'BAC_RESULT VALUE',
#    'CELL_PHONE_USE',
]]
print(df_people_drop.shape)

In [ ]:
# Filtering the data with Drivers only
df_people_driver = df_people_drop[df_people_drop.PERSON_TYPE == 'DRIVER']
print(df_people_driver.shape)

In [ ]:
df_people_driver.isna().sum()

In [ ]:
df_people_driver.describe()

In [ ]:
# Dropping rows with driver age bigger than 90 and below 18
df_people_driver_age = df_people_driver[(df_people_driver['AGE'] >= 18) & (df_people_driver['AGE'] <= 90) | (df_people_driver['AGE'].isna())]

In [ ]:
# Replacing all UNKNOWN values to missing values
#Unknown = ['UNKNOWN','USAGE UNKNOWN', 'DEPLOYMENT UNKNOWN']
df_vehicles_w_occ.replace({'Unknown' :np.nan , 'USAGE UNKNOWN' :np.nan , 'DEPLOYMENT UNKNOWN':np.nan }, inplace=True)

In [ ]:
merged = df_crashes_drop.merge(df_vehicles_w_occ, on='CRASH_RECORD_ID')
print(merged.shape)

In [ ]:
df = merged.merge(df_people_driver_age, on=['VEHICLE_ID','CRASH_RECORD_ID'])
print(df.shape)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# creating bins for times 
# 0-6 = Late Night/Early Morning 
# 6-12 = Morning 
# 12-18 = Afternoon/Rush Hour
# 18-23 = Late Evening 
df['time_bins'] = pd.cut(x=df['CRASH_HOUR'], bins = [-1,6,12,18,24], 
                         labels = ['Late Night/Early Morning', 
                        'Morning', 'Afternoon/Rush Hour','Late Evening'])
df.head(50)

In [ ]:
# Dropping Unnecassary Features
df_relv = df.drop([
    'CRASH_RECORD_ID',
#    'RD_NO',
#    'PERSON_ID',
    'CRASH_DATE',
    'VEHICLE_ID',
#    'CITY',
#    'STATE',
#    'ZIPCODE',
    'PERSON_TYPE',
    'OCCUPANT_CNT',
#    'has_injuries',
#    'has_fatality',
#    'LONGITUDE',
#    'LATITUDE',
    'ROAD_DEFECT',
#    'LIC_PLATE_STATE',
#    'TRAVEL_DIRECTION', 
#    'DRIVERS_LICENSE_STATE',
#    'INJURY_CLASSIFICATION',
#    'DRIVER_ACTION',
#    'PHYSICAL_CONDITION'],
],axis=1)

In [ ]:
df_drop_missing = df_relv.dropna(subset = ['LATITUDE','LONGITUDE'], axis = 0).reset_index(drop=True)
df_drop_missing.shape

In [ ]:
df_drop_missing.head()

In [ ]:
df1 = df_drop_missing[['LATITUDE',"LONGITUDE"]]
df1

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 50, init ='k-means++')
kmeans.fit(df1[df1.columns]) # Compute k-means clustering.
df1['cluster_label'] = kmeans.fit_predict(df1[df1.columns])
#centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
#labels = kmeans.predict(df1[df1.columns]) # Labels of each point
df1.head(20)

In [ ]:
#sns.scatterplot(x=df1.LATITUDE, y = df1.LONGITUDE, hue=df1.cluster_label, palette = 'rocket')

In [ ]:
plt.subplots(figsize=(10,10))
fg = sns.scatterplot(data=df1, x='LONGITUDE',y='LATITUDE', palette = 'viridis',
                hue='cluster_label', size_norm=(1,100), alpha=.4)
fg.set_axis_off()
fg.set_title('Crashes in Chicago',fontweight='bold');

In [ ]:
#df1.plot.scatter(x = 'LATITUDE', y = 'LONGITUDE', c=df1.cluster_label, s=50, cmap='viridis')
#plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)



In [ ]:
df_drop_missing.isna().sum()

In [ ]:
df_drop_missing2 = df_drop_missing.dropna(subset= ['AGE','VEHICLE_YEAR','ROADWAY_SURFACE_COND','WEATHER_CONDITION'], axis = 0).reset_index(drop=True)
print(df_drop_missing2.isna().sum())
print(df_drop_missing2.shape)

In [ ]:
# Splitting features into numeric and categorical
numeric_columns = [
    'POSTED_SPEED_LIMIT',
#    'NUM_UNITS',
    'VEHICLE_YEAR',
#    'OCCUPANT_CNT',
    'AGE',
    
]

cat_columns = [
    'WEATHER_CONDITION', 
    'ROADWAY_SURFACE_COND',  
#    'ROAD_DEFECT',  
#    'CRASH_HOUR',  
    'CRASH_DAY_OF_WEEK',  
    'CRASH_MONTH',   
#    'ZIPCODE',  
    'SEX',  
    'SAFETY_EQUIPMENT', 
    'AIRBAG_DEPLOYED',  
    'time_bins',
]


Target1 = df_drop_missing2[[
    'has_injuries',           
#    'has_fatality',    
]]

Target2 = df_drop_missing2[[
#    'has_injuries',           
    'has_fatality',    
]]

numeric_df = df_drop_missing2[numeric_columns]
cat_df = df_drop_missing2[cat_columns]

# Train_Test Split

In [ ]:
X = pd.concat([numeric_df,cat_df] , axis = 1)
y = Target1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 42)

X_train_nums = X_train [[
    'POSTED_SPEED_LIMIT',
    'VEHICLE_YEAR',
    'AGE',
    
]]

X_train_cats = X_train [[  
    'WEATHER_CONDITION', 
    'ROADWAY_SURFACE_COND',  
#    'ROAD_DEFECT',  
#    'CRASH_HOUR',  
    'CRASH_DAY_OF_WEEK',  
    'CRASH_MONTH',   
#    'ZIPCODE',  
    'SEX',  
    'SAFETY_EQUIPMENT', 
    'AIRBAG_DEPLOYED',  
    'time_bins',
]]

X_train = pd.concat([X_train_nums,X_train_cats] , axis = 1)

In [ ]:
# Cheking the distribution of targets

y_train.value_counts(normalize=True)

# Creating Pipeline for Each Model

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline

In [ ]:
# Scaling and One Hot Encoding 
numerical_pipeline = Pipeline(steps=[
    ('ss', StandardScaler())
])
                
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first',
                         sparse=False))
])

trans = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, X_train_nums.columns),
    ('categorical', categorical_pipeline, X_train_cats.columns)
])

# Under & Over Sampling    
over = SMOTE(sampling_strategy = 0.7)
under = RandomUnderSampler(sampling_strategy = 0.8)

In [ ]:
# Logistic Regression Model with Pipeline
LR_pipe = imbPipeline(steps=[
    ('trans', trans),
    ('over' , over),
    ('under', under),
    ('LR', LogisticRegression(max_iter = 1e3, penalty = 'none', random_state = 42))
])

# Create the GridSearchCV object with different hyperparameters
parameters_LR = {
    'LR__C': [1, 10, 100],
    'LR__solver': ['lbfgs','sag', 'saga']
}

cv_LR = GridSearchCV(LR_pipe, param_grid=parameters_LR)

cv_LR.fit(X_train, y_train)

print(y_train.value_counts(normalize=True))

# Predict the label with the best model
y_pred_LR = cv_LR.predict(X_test)
print(y_pred_LR)

In [ ]:
# DecisionTree Model with Pipeline
DT_pipe = imbPipeline(steps=[
    ('trans', trans),
    ('over' , over),
    ('under', under),
    ('DT', DecisionTreeClassifier(random_state = 42))
])

parameters_DT = {
    'DT__max_depth': [2, 4, 6, 8],
    'DT__min_samples_split': [5, 10, 15],
    'DT__criterion': ['gini', 'entropy']
}

cv_DT = GridSearchCV(DT_pipe, param_grid=parameters_DT)

cv_DT.fit(X_train, y_train)

# Predict the label with the best model
y_pred_DT = cv_DT.predict(X_test)
print(y_pred_DT)

In [ ]:
# KNN Model with Pipeline
KNN_pipe = imbPipeline(steps=[
    ('trans', trans),
    ('over' , over),
    ('under', under),
    ('KNN', KNeighborsClassifier(random_state = 42))
])

# Create the GridSearchCV object with different hyperparameters
parameters_KNN = {
    'KNN__n_neighbors': [1, 3, 5, 7],
    'KNN__metric': ['minkowski', 'manhattan'],
    'KNN__weights': ['uniform', 'distance'],
}

cv_KNN = GridSearchCV(KNN_pipe, param_grid=parameters_KNN)

cv_KNN.fit(X_train, y_train)

# Predict the label with the best model
y_pred_KNN = cv_KNN.predict(X_test)
print(y_pred_KNN)

In [ ]:
# Random Forrest Model with Pipeline
RF_pipe = imbPipeline(steps=[
    ('trans', trans),
    ('over' , over),
    ('under', under),
    ('RF', RandomForestClassifier(random_state = 42))
])

# Create the GridSearchCV object with different hyperparameters
parameters = {
    'RF__n_estimators': [10, 100, 1000],
    'RF__max_depth': [2, 4, 6, 8],
    'RF__min_samples_split': [5, 10, 15],
    'RF__criterion': ['gini', 'entropy']
}

cv_RF = GridSearchCV(RF_pipe, param_grid=parameters_RF)

cv_RF.fit(X_train, y_train)

# Predict the label with the best model
y_pred_RF = cv_RF.predict(X_test)
print(y_pred_RF)
